# Streaming
Methods: .stream() and astream()

- These methods are sync and async methods for streaming back results.

Additional parameters in streaming modes for graph state

- values: This streams the full state of the graph after each node is called.
- updates: This streams updates to the state of the graph after each node is called

In [45]:
from langgraph.graph import StateGraph, START,END
from typing import TypedDict,Annotated
from langgraph.graph.message import add_messages
from langchain_core.messages import AnyMessage, HumanMessage
from dotenv import load_dotenv
load_dotenv()

True

In [46]:
class state(TypedDict):
    messages: Annotated[list[AnyMessage],add_messages]

In [47]:
from langchain_groq import ChatGroq
llm=ChatGroq(model='llama-3.1-8b-instant')

In [48]:
def gen_message(state:state)->state:
    return {'messages':llm.invoke(state['messages'])}

In [49]:
from langgraph.checkpoint.memory import MemorySaver
# graph
graph=StateGraph(state)

graph.add_node('chatbot',gen_message)
graph.add_edge(START,'chatbot')
graph.add_edge('chatbot',END)
checkpoint=MemorySaver()
workflow=graph.compile(checkpoint)

In [50]:
config={
    'configurable':{
        'thread_id':'1'
    }
}

In [51]:
workflow.invoke({
    'messages':'hi how are you',
},
config=config)

{'messages': [HumanMessage(content='hi how are you', additional_kwargs={}, response_metadata={}, id='87e7d2fc-8cc3-44d7-a076-5091b90897e6'),
  AIMessage(content="I'm just a computer program, so I don't have feelings, but I'm functioning properly and ready to help you with any questions or tasks you may have. How can I assist you today?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 39, 'total_tokens': 80, 'completion_time': 0.048587475, 'prompt_time': 0.030003999, 'queue_time': 0.204643392, 'total_time': 0.078591474}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_2115512ff6', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--c1b9cf10-8399-4b56-99e6-972bd6a2b542-0', usage_metadata={'input_tokens': 39, 'output_tokens': 41, 'total_tokens': 80})]}

In [52]:
for chunk in workflow.stream({'messages':'my name is talha and i like coding'},config,stream_mode='update'):
    
    print(chunk)

In [53]:
for chunk in workflow.stream({'messages':"I also like football"},config,stream_mode="values"):
    print(chunk)

{'messages': [HumanMessage(content='hi how are you', additional_kwargs={}, response_metadata={}, id='87e7d2fc-8cc3-44d7-a076-5091b90897e6'), AIMessage(content="I'm just a computer program, so I don't have feelings, but I'm functioning properly and ready to help you with any questions or tasks you may have. How can I assist you today?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 39, 'total_tokens': 80, 'completion_time': 0.048587475, 'prompt_time': 0.030003999, 'queue_time': 0.204643392, 'total_time': 0.078591474}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_2115512ff6', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--c1b9cf10-8399-4b56-99e6-972bd6a2b542-0', usage_metadata={'input_tokens': 39, 'output_tokens': 41, 'total_tokens': 80}), HumanMessage(content='my name is talha and i like coding', additional_kwargs={}, response_metadata={}, id='cd026b77-8c8e-4d5f-8f5b-f49ae02d5bf4')

In [54]:
for chunk in workflow.stream({'messages':'my name is talha and i also like cycling'},config,stream_mode='update'):
    
    print(chunk)

# Streaming The Responses With astream Method

Streaming tokens We often want to stream more than graph state.
In particular, with chat model calls it is common to stream the tokens as they are generated.

We can do this using the astream_events method, which streams back events as they happen inside nodes!

Each event is a dict with a few keys:
- event: This is the type of event that is being emitted.
- name: This is the name of event.
- data: This is the data associated with the event.
- metadata: Containslanggraph_node, the node emitting the event.

In [55]:
config={
    'configurable':{
        'thread_id':'2'
    }
}

In [61]:
async for event in workflow.astream_events({'messages':['My Name is talha and your']},config,version='v2'):

    print(event)

{'event': 'on_chain_start', 'data': {'input': {'messages': ['My Name is talha and your']}}, 'name': 'LangGraph', 'tags': [], 'run_id': 'a6c5677c-dd6d-4c89-9b6c-1ee46fe04921', 'metadata': {'thread_id': '2'}, 'parent_ids': []}
{'event': 'on_chain_start', 'data': {'input': {'messages': [HumanMessage(content='My Name is talha and your', additional_kwargs={}, response_metadata={}, id='a6f6d1da-f9d3-4057-89a2-85bcdff97eac')]}}, 'name': 'chatbot', 'tags': ['graph:step:1'], 'run_id': '2aeed44d-d25a-401c-b5e3-92ce989b647d', 'metadata': {'thread_id': '2', 'langgraph_step': 1, 'langgraph_node': 'chatbot', 'langgraph_triggers': ('branch:to:chatbot',), 'langgraph_path': ('__pregel_pull', 'chatbot'), 'langgraph_checkpoint_ns': 'chatbot:94e9b5bb-ac97-803d-7e8c-51c1d8df2f83'}, 'parent_ids': ['a6c5677c-dd6d-4c89-9b6c-1ee46fe04921']}
{'event': 'on_chat_model_start', 'data': {'input': {'messages': [[HumanMessage(content='My Name is talha and your', additional_kwargs={}, response_metadata={}, id='a6f6d1da